# 암환자 유전체 데이터 기반 암종 분류 AI 모델 개발


- '2024 생명연구자원 AI활용 경진대회'는 바이오 데이터를 기반으로 한 AI 기술의 문제 해결 능력을 탐구하는 것을 목표로 합니다. <br>이 대회는 바이오 분야에서 AI 활용의 저변을 확대하고, 복잡한 바이오 데이터를 효율적으로 분석 및 해석할 수 있는 AI 알고리즘 개발에 초점을 맞추고 있습니다. <br><br>
- 본 대회의 구체적인 과제는 암환자 유전체 데이터의 변이 정보를 활용하여 암종을 분류하는 AI 모델을 개발하는 것입니다. <br>참가자들은 제공된 학습 데이터셋(암환자 유전체 변이 정보)을 사용하여 특정 변이 정보를 바탕으로 암종을 정확하게 분류할 수 있는 AI 알고리즘을 개발해야 합니다. <br><br>
- 이 대회의 궁극적인 목적은 바이오 데이터의 활용도를 높이고, 바이오 분야에서 AI 기술의 적용 가능성을 극대화하며, 인공지능 기술이 실제 바이오 의료 문제 해결에 어떻게 기여할 수 있는지 탐구하는 것입니다.

# VERSIONS #
### GPU 0 : NVIDIA GeForce RTX 3080 Ti Laptop GPU
### GPU 1 : AMD Radeon(TM) Graphics
### CPU : AMD Ryzen 9 6900HX with Radeon Graphics

### -------------------------- Python & library version --------------------------
### Python version: 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
### pandas version: 2.0.3
### numpy version: 1.21.5
### matplotlib version: 3.5.2
### tqdm version: 4.64.1
### xgboost version: 1.7.2
### lightgbm version: 3.3.3
### catboost version: 1.1.1
### scikit-learn version: 1.0.2
------------------------------------------------------------------------------

In [6]:
import pandas as pd
import numpy as np
import matplotlib
import sklearn  # scikit-learn 임포트
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import xgboost as xgb
from tqdm import tqdm  # tqdm 임포트
import warnings
import pandas as pd
import re

# PerformanceWarning을 무시하도록 설정
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

# Load Data

In [29]:
# KIPAN, STES 빼고 24개 증강

In [30]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [31]:
import re

def convert_format(data):
    # 패턴 수정: 마지막 그룹에서 변이 문자를 처리하기 위해 \w* 사용
    pattern = r"(\d+)_(\d+)(\w+)\>(\w*)"
    
    # 결과를 저장할 리스트
    result = []
    
    # 데이터를 공백 기준으로 분리 (변이가 여러 개일 수 있으므로)
    parts = data.split()

    for part in parts:
        # 정규 표현식으로 변이를 추출
        match = re.match(pattern, part)
        if match:
            # 각각의 위치와 아미노산 정보 추출
            pos1, pos2, original_aa, new_aa = match.groups()
            
            # 변이 형식으로 변환
            aa1 = f"{original_aa[0]}{pos1}{new_aa[0] if new_aa else original_aa[0]}"
            aa2 = f"{original_aa[1]}{pos2}{new_aa[1] if new_aa and len(new_aa) > 1 else original_aa[1]}"
            
            # 변환된 두 아미노산을 결과에 추가
            result.append(f"{aa1} {aa2}")
        else:
            # F1917F 같은 형식은 그대로 추가
            result.append(part)
    
    # 변환된 데이터 반환 (공백으로 다시 결합)
    return ' '.join(result)


import re

def convert_deletion_format(data):
    # 패턴: I255_I258del 같은 형식을 찾는 정규식
    pattern = r"([A-Z])(\d+)_([A-Z])(\d+)del"

    # 결과를 저장할 리스트
    result = []
    
    # 데이터를 공백 기준으로 분리 (여러 변이가 있을 수 있으므로)
    parts = data.split()

    for part in parts:
        # 정규 표현식으로 변이를 추출
        match = re.match(pattern, part)
        if match:
            # 아미노산과 위치 정보 추출
            aa1, pos1, aa2, pos2 = match.groups()
            
            # 각 변이를 단일 형식으로 변환
            result.append(f"{aa1}{pos1}del")
            result.append(f"{aa2}{pos2}del")
        else:
            # 단일 형식 (예: I410del)인 경우 그대로 추가
            result.append(part)
    
    # 변환된 데이터 반환 (공백으로 결합)
    return ' '.join(result)
# 테스트 예시
# data = '1099_1100PL>PL F1917F'

# ACC

In [35]:
import pandas as pd
import gzip

# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/ACC.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분

df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['One_Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['One_Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['One_Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['One_Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['One_Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['One_Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['One_Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA_0 = df_TCGA.drop_duplicates()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\3946466048.py:9: DtypeWarning: Columns (74,78,114,130) have mixed types. Specify dtype option on import or set low_memory=False.
  df_TCGA = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분


In [36]:
import tarfile

# tar.gz 파일 열기
with tarfile.open('additional_data_2/ACC/acc_tcga.tar.gz', 'r:gz') as tar:
    # 압축 파일 내부의 파일 목록 보기
#     tar.list()

    # 원하는 파일을 선택하여 읽기
    file = tar.extractfile('acc_tcga/data_mutations.txt')
    df_TCGA_1 = pd.read_csv(file, sep='\t')  # 탭으로 구분된 텍스트 파일 읽기

In [37]:
import tarfile

# tar.gz 파일 열기
with tarfile.open('additional_data_2/ACC/acc_tcga_gdc.tar.gz', 'r:gz') as tar:
    # 압축 파일 내부의 파일 목록 보기
#     tar.list()

    # 원하는 파일을 선택하여 읽기
    file = tar.extractfile('acc_tcga_gdc/data_mutations.txt')
    df_TCGA_2 = pd.read_csv(file, sep='\t', skiprows=2)  # 탭으로 구분된 텍스트 파일 읽기

In [38]:
import tarfile

# tar.gz 파일 열기
with tarfile.open('additional_data_2/ACC/acc_tcga_pan_can_atlas_2018.tar.gz', 'r:gz') as tar:
    # 압축 파일 내부의 파일 목록 보기
#     tar.list()

    # 원하는 파일을 선택하여 읽기
    file = tar.extractfile('acc_tcga_pan_can_atlas_2018/data_mutations.txt')
    df_TCGA_3 = pd.read_csv(file, sep='\t', skiprows=0)  # 탭으로 구분된 텍스트 파일 읽기

In [39]:
df_TCGA_1 = df_TCGA_1[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA_2 = df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA_3 = df_TCGA_3[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA_1,df_TCGA_2,df_TCGA_3,df_TCGA_0],axis=0)

In [40]:
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]

tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'ACC'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'ACC'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|████████████████████████████████████████████████████████████████████████████████| 265/265 [00:58<00:00,  4.52it/s]


# BLCA

In [41]:
import pandas as pd
import gzip

# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/BLCA.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분

# df_TCGA = pd.concat([df_TCGA_1,df_TCGA_2,df_TCGA_3,df_TCGA_4],axis=0)#     .drop_duplicates()

df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['One_Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['One_Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['One_Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['One_Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['One_Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['One_Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['One_Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA_0 = df_TCGA.drop_duplicates()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\20855946.py:9: DtypeWarning: Columns (120) have mixed types. Specify dtype option on import or set low_memory=False.
  df_TCGA = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분


In [42]:
import tarfile

# tar.gz 파일 열기
with tarfile.open('additional_data_2/BLCA/blca_bgi.tar.gz', 'r:gz') as tar:
    # 압축 파일 내부의 파일 목록 보기
#     tar.list()

    # 원하는 파일을 선택하여 읽기
    file = tar.extractfile('blca_bgi/data_mutations.txt')
    df_TCGA_1 = pd.read_csv(file, sep='\t')  # 탭으로 구분된 텍스트 파일 읽기

In [43]:
import tarfile

# tar.gz 파일 열기
with tarfile.open('additional_data_2/BLCA/blca_dfarber_mskcc_2014.tar.gz', 'r:gz') as tar:
    # 압축 파일 내부의 파일 목록 보기
#     tar.list()

    # 원하는 파일을 선택하여 읽기
    file = tar.extractfile('blca_dfarber_mskcc_2014/data_mutations.txt')
    df_TCGA_2 = pd.read_csv(file, sep='\t',skiprows=1)  # 탭으로 구분된 텍스트 파일 읽기

In [44]:
import tarfile

# tar.gz 파일 열기
with tarfile.open('additional_data_2/BLCA/blca_tcga.tar.gz', 'r:gz') as tar:
    # 압축 파일 내부의 파일 목록 보기
#     tar.list()

    # 원하는 파일을 선택하여 읽기
    file = tar.extractfile('blca_tcga/data_mutations.txt')
    df_TCGA_3 = pd.read_csv(file, sep='\t',skiprows=0)  # 탭으로 구분된 텍스트 파일 읽기

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\2966922366.py:10: DtypeWarning: Columns (50,58,61,71,77,94) have mixed types. Specify dtype option on import or set low_memory=False.
  df_TCGA_3 = pd.read_csv(file, sep='\t',skiprows=0)  # 탭으로 구분된 텍스트 파일 읽기


In [45]:
import tarfile

# tar.gz 파일 열기
with tarfile.open('additional_data_2/BLCA/blca_tcga_gdc.tar.gz', 'r:gz') as tar:
    # 압축 파일 내부의 파일 목록 보기
#     tar.list()

    # 원하는 파일을 선택하여 읽기
    file = tar.extractfile('blca_tcga_gdc/data_mutations.txt')
    df_TCGA_4 = pd.read_csv(file, sep='\t',skiprows=2)  # 탭으로 구분된 텍스트 파일 읽기
df_TCGA_4.head(1)

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Classification,...,HGVSc,HGVSp,HGVSp_Short,Transcript_ID,RefSeq,Protein_position,Codons,Exon_Number,genomic_location_explanation,Annotation_Status
0,UTS2,10911.0,BI,GRCh38,1,7847808,7847808,+,missense_variant,Missense_Mutation,...,ENST00000361696.10:c.333G>C,p.Lys111Asn,p.K111N,ENST00000361696,NM_006786.4,111.0,aaG/aaC,4/4,NaN,SUCCESS


In [46]:
import tarfile

# tar.gz 파일 열기
with tarfile.open('additional_data_2/BLCA/blca_tcga_pan_can_atlas_2018.tar.gz', 'r:gz') as tar:
    # 압축 파일 내부의 파일 목록 보기
#     tar.list()

    # 원하는 파일을 선택하여 읽기
    file = tar.extractfile('blca_tcga_pan_can_atlas_2018/data_mutations.txt')
    df_TCGA_5 = pd.read_csv(file, sep='\t',skiprows=0)  # 탭으로 구분된 텍스트 파일 읽기
df_TCGA_5.head(1)

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Classification,...,SYMBOL_SOURCE,TREMBL,TSL,UNIPARC,VARIANT_CLASS,all_effects,cDNA_position,n_depth,t_depth,Annotation_Status
0,DNMBP,23268.0,.,GRCh37,10,101715548,101715548,+,synonymous_variant,Silent,...,HGNC,B4E0Q3_HUMAN,.,UPI000013D6C9,SNV,"DNMBP,synonymous_variant,p.%3D,ENST00000324109...",1775,88,96,SUCCESS


In [47]:
import tarfile

# tar.gz 파일 열기
with tarfile.open('additional_data_2/BLCA/blca_tcga_pub.tar.gz', 'r:gz') as tar:
    # 압축 파일 내부의 파일 목록 보기
#     tar.list()

    # 원하는 파일을 선택하여 읽기
    file = tar.extractfile('blca_tcga_pub/data_mutations.txt')
    df_TCGA_6 = pd.read_csv(file, sep='\t',skiprows=0)  # 탭으로 구분된 텍스트 파일 읽기
df_TCGA_6.head(1)

,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Consequence,Variant_Classification,...,n_ref_count,n_alt_count,HGVSc,HGVSp,HGVSp_Short,Transcript_ID,RefSeq,Protein_position,Codons,Hotspot
0,EMC1,0,broad.mit.edu,GRCh37,1,19549914,19549914,+,synonymous_variant,Silent,...,NaN,NaN,ENST00000477853.1:c.2352C>T,p.Ile784=,p.I784=,ENST00000477853,NM_001271427.1,784.0,atC/atT,0


In [48]:
df_TCGA_1 = df_TCGA_1[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA_2 = df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA_3 = df_TCGA_3[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA_4 = df_TCGA_4[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA_5 = df_TCGA_5[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA_6 = df_TCGA_6[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA_1,df_TCGA_2,df_TCGA_3,df_TCGA_4,df_TCGA_5,df_TCGA_6,df_TCGA_0],axis=0)

In [49]:
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]

tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'BLCA'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'BLCA'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|██████████████████████████████████████████████████████████████████████████████| 1379/1379 [05:39<00:00,  4.06it/s]


# additional data

In [50]:
train

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R895R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7840,ID,BLCA,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
7841,ID,BLCA,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
7842,ID,BLCA,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
7843,ID,BLCA,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,I180T,WT,WT,S276F,WT,WT,WT,WT,WT


# BRCA

In [51]:
import os
import tarfile
import pandas as pd

# BRCA 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/BRCA'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도

    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#version'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError as e:
            print(f"ParserError encountered: {e}")
            return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                df = df[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
                # df가 제대로 로드되었는지 확인 후 리스트에 추가
                if df is not None:
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출
                
df_TCGA = pd.concat(df_list, ignore_index=True)

###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/BRCA.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################

df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

'#version' detected with skiprows=0, retrying with skiprows=1


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\820952279.py:18: DtypeWarning: Columns (1,5,6,14,15,20,21,22,23,24,27,28,30,31,32,42,44,45,46,48,49,50,52,53,55,56,57,58,59,60,61,63,67,70,71,72,73,75,77,82,83,84,88,91,93,94,96,97,99,101,105,107,110,113,114,117,120,121,122,125,126,127,128,129,130,132,135,138,140,142,145,147,149,151,152) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


'#version' detected with skiprows=0, retrying with skiprows=1


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\820952279.py:18: DtypeWarning: Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\820952279.py:18: DtypeWarning: Columns (143,144) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\820952279.py:60: DtypeWarning: Columns (120) have mixed types. Specify dtype option on import or set low_memory=False.
  df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분


In [52]:
df_TCGA['Tumor_Sample_Barcode'].value_counts()

Tumor_Sample_Barcode
TCGA-AN-A046-01                 7023
TCGA-AC-A23H-01                 5844
TCGA-AN-A046-01A-21W-A050-09    5678
TCGA-AC-A23H-01A-11D-A159-09    4745
TCGA-EW-A2FV-01                 3751
                                ... 
TCGA-EW-A1P1-01A-31D-A14G-09       2
TCGA-AO-A1KO-01A-31D-A188-09       2
TCGA-A7-A0DC-01                    1
SA085                              1
SA063                              1
Name: count, Length: 2346, dtype: int64

In [53]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'BRCA'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'BRCA'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

#980개 

100%|██████████████████████████████████████████████████████████████████████████████| 2336/2336 [10:59<00:00,  3.54it/s]


# additional data

# CESC

In [54]:
import os
import tarfile
import pandas as pd

# BRCA 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/CESC'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#version'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                df = df[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
                # df가 제대로 로드되었는지 확인 후 리스트에 추가
                if df is not None:
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출
                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/CESC.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1968593471.py:18: DtypeWarning: Columns (52,53,56,72,77,81,95,100,101,103) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


ParserError encountered with skiprows=0, retrying with skiprows=2


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1968593471.py:18: DtypeWarning: Columns (4,38,39,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


In [55]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'CESC'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'CESC'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

# 288개

100%|████████████████████████████████████████████████████████████████████████████████| 873/873 [04:45<00:00,  3.06it/s]


In [56]:
train

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R895R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11049,ID,CESC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
11050,ID,CESC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
11051,ID,CESC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
11052,ID,CESC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


# additional data

# COAD

In [57]:
import os
import tarfile
import pandas as pd

# BRCA 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/COAD'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#version'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                df = df[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
                # df가 제대로 로드되었는지 확인 후 리스트에 추가
                if df is not None:
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출
                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/COAD.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

ParserError encountered with skiprows=0, retrying with skiprows=2
ParserError encountered with skiprows=0, retrying with skiprows=2


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\2044667432.py:64: DtypeWarning: Columns (120) have mixed types. Specify dtype option on import or set low_memory=False.
  df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분


In [58]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'COAD'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'COAD'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|██████████████████████████████████████████████████████████████████████████████| 1013/1013 [06:00<00:00,  2.81it/s]


# additional data

# DLBC

In [59]:
import os
import tarfile
import pandas as pd

# BRCA 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/DLBC'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#version'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                df = df[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
                # df가 제대로 로드되었는지 확인 후 리스트에 추가
                if df is not None:
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출
                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/DLBC.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

In [60]:
df_TCGA

,Tumor_Sample_Barcode,Hugo_Symbol,Consequence,HGVSp_Short
0,TCGA-RQ-A6JB-01,C10orf71,inframe_deletion,p.F1299del
1,TCGA-RQ-A6JB-01,STAT3,inframe_deletion,p.E616del
2,TCGA-RQ-A6JB-01,KRTAP4-5,inframe_insertion,p.C81_Q82insHPSCC
3,TCGA-RQ-A6JB-01,DMKN,inframe_insertion,p.G270_G271insSSS
4,TCGA-RQ-A6JB-01,B2M,missense_variant,p.D54V
...,...,...,...,...
6624,TCGA-FA-A4XK-01A-11D-A31X-10,IGLV4-69,missense_variant,p.S72I
6625,TCGA-FA-A4XK-01A-11D-A31X-10,IGLC2,inframe_deletion,p.S62_Y66delinsN
6626,TCGA-FA-A4XK-01A-11D-A31X-10,IGLC2,synonymous_variant,p.T90T
6627,TCGA-FA-A4XK-01A-11D-A31X-10,IGLC3,inframe_deletion,p.S62_Y66delinsN


In [61]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'DLBC'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'DLBC'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 94/94 [00:35<00:00,  2.64it/s]


# additional data

# GBMLGG

In [62]:
import tarfile

# tar.gz 파일 열기
with tarfile.open('additional_data_2/GBMLGG/glioma_msk_2018.tar.gz', 'r:gz') as tar:
    # 압축 파일 내부의 파일 목록 보기
#     tar.list()

    # 원하는 파일을 선택하여 읽기
    file = tar.extractfile('glioma_msk_2018/data_mutations.txt')
    df_TCGA_1 = pd.read_csv(file, sep='\t',skiprows=0)  # 탭으로 구분된 텍스트 파일 읽기

In [63]:
df_TCGA_1 = df_TCGA_1[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','Amino_Acid_Change']]


In [64]:
import tarfile

# tar.gz 파일 열기
with tarfile.open('additional_data_2/GBMLGG/glioma_mskcc_2019.tar.gz', 'r:gz') as tar:
    # 압축 파일 내부의 파일 목록 보기
#     tar.list()

    # 원하는 파일을 선택하여 읽기
    file = tar.extractfile('glioma_mskcc_2019/data_mutations.txt')
    df_TCGA_2 = pd.read_csv(file, sep='\t',skiprows=2)  # 탭으로 구분된 텍스트 파일 읽기

In [65]:
df_TCGA_2 = df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]


In [66]:
df_TCGA_1.columns = ['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']

In [67]:
df_TCGA = pd.concat([df_TCGA_1,df_TCGA_2])

###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/GBM.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################

In [68]:
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

In [69]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'GBMLGG'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'GBMLGG'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|██████████████████████████████████████████████████████████████████████████████| 1470/1470 [09:11<00:00,  2.67it/s]


# additional data

# HNSC

In [70]:
import os
import tarfile
import pandas as pd

# BRCA 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/HNSC'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#version'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                df = df[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
                # df가 제대로 로드되었는지 확인 후 리스트에 추가
                if df is not None:
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출
                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/HNSC.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1262893719.py:18: DtypeWarning: Columns (1,5,6,20,21,22,23,24,30,31,42,44,45,46,48,53,54,55,57,60,64,68,69,70,71,81,88,99,102,103,106,116,119,125,129,133) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


'#version' detected with skiprows=0, retrying with skiprows=1


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1262893719.py:18: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1262893719.py:18: DtypeWarning: Columns (67) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


ParserError encountered with skiprows=0, retrying with skiprows=2


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1262893719.py:18: DtypeWarning: Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1262893719.py:18: DtypeWarning: Columns (123) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


In [71]:
df_TCGA

,Tumor_Sample_Barcode,Hugo_Symbol,Consequence,HGVSp_Short
0,HN_0-046,ASIC2,synonymous_variant,p.A402A
1,HN_0-046,ADAMTS18,missense_variant,p.L568S
2,HN_0-046,ADAMTS9,stop_gained,p.G379*
3,HN_0-046,AMFR,missense_variant,p.R414C
4,HN_0-046,ANKAR,missense_variant,p.K882T
...,...,...,...,...
87962,TCGA-D6-A74Q-01A-11D-A34J-08,RNF128,missense_variant,p.N74I
87963,TCGA-D6-A74Q-01A-11D-A34J-08,TRPC5,missense_variant,p.R685T
87964,TCGA-D6-A74Q-01A-11D-A34J-08,LRCH2,missense_variant,p.L716F
87965,TCGA-D6-A74Q-01A-11D-A34J-08,BRS3,missense_variant,p.S386L


In [72]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'HNSC'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'HNSC'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

#  507개

100%|██████████████████████████████████████████████████████████████████████████████| 1632/1632 [11:01<00:00,  2.47it/s]


In [272]:
# 여기까지완료

# additional data

# KIRC

In [73]:
import os
import tarfile
import pandas as pd

# KIRC 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/KIRC'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]
gz_files = gz_files[1:]
# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#version'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                
                # df가 None이 아닌 경우에만 계속 처리
                if df is not None:
                    # Amino_Acid_Change 열이 있는지 확인하고, 있으면 이름 변경
                    if 'Amino_Acid_Change' in df.columns:
                        df.rename(columns={'Amino_Acid_Change': 'HGVSp_Short'}, inplace=True)
                    
                    # 필요한 열만 선택
                    df = df[['Tumor_Sample_Barcode', 'Hugo_Symbol', 'Consequence', 'HGVSp_Short']]
                    
                    # df_list에 추가
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출
                
df_TCGA = pd.concat(df_list, ignore_index=True)

In [74]:
import tarfile

# tar.gz 파일 열기
with tarfile.open('additional_data_2/KIRC/ccrcc_irc_2014.tar.gz', 'r:gz') as tar:
    # 압축 파일 내부의 파일 목록 보기
    tar.list()

    # 원하는 파일을 선택하여 읽기
    file = tar.extractfile('ccrcc_irc_2014/data_mutations.txt')
    df_TCGA_irc = pd.read_csv(file, sep='\t',skiprows=1)  # 탭으로 구분된 텍스트 파일 읽기
df_TCGA_irc.head(3)
df_TCGA_irc = df_TCGA_irc[['Tumor_Sample_Barcode', 'Hugo_Symbol', 'Consequence', 'HGVSp_Short']]


?rwxr-xr-x runner/docker          0 2024-08-13 04:14:26 ccrcc_irc_2014/ 
?rw-r--r-- runner/docker    3910113 2024-08-13 04:59:09 ccrcc_irc_2014/data_mutations.txt 
?rw-r--r-- runner/docker        455 2024-08-13 04:14:26 ccrcc_irc_2014/LICENSE 
?rw-r--r-- runner/docker        869 2024-08-13 04:59:09 ccrcc_irc_2014/data_clinical_patient.txt 
?rw-r--r-- runner/docker        141 2024-08-13 04:14:26 ccrcc_irc_2014/meta_clinical_sample.txt 
?rw-r--r-- runner/docker        310 2024-08-13 04:14:26 ccrcc_irc_2014/meta_study.txt 
?rw-r--r-- runner/docker        143 2024-08-13 04:14:26 ccrcc_irc_2014/meta_clinical_patient.txt 
?rwxr-xr-x runner/docker          0 2024-08-13 04:14:26 ccrcc_irc_2014/case_lists/ 
?rw-r--r-- runner/docker        964 2024-08-13 04:14:26 ccrcc_irc_2014/case_lists/cases_sequenced.txt 
?rw-r--r-- runner/docker        927 2024-08-13 04:14:26 ccrcc_irc_2014/case_lists/cases_all.txt 
?rw-r--r-- runner/docker        332 2024-08-13 04:14:26 ccrcc_irc_2014/meta_mutations.txt 
?

In [75]:
df_TCGA = pd.concat([df_TCGA,df_TCGA_irc]).reset_index(drop=True)

In [76]:
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/KIRC.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\4228264068.py:7: DtypeWarning: Columns (120) have mixed types. Specify dtype option on import or set low_memory=False.
  df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분


In [77]:
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

In [78]:
df_TCGA

,Tumor_Sample_Barcode,Hugo_Symbol,Consequence,HGVSp_Short
1,K29,ABCB8,inframe_deletion,p.L637_D638del
2,K44,ABTB2,missense_variant,p.S845A
3,K20,ACSM5,synonymous_variant,p.N409N
4,K38,ADAM11,missense_variant,p.V245A
5,K20,ADCY9,missense_variant,p.H239Y
...,...,...,...,...
25717,TCGA-CZ-4853-01A-01D-1429-08,KIAA0930,missense_variant,p.D86Y
25718,TCGA-CZ-4853-01A-01D-1429-08,TLR8,missense_variant,p.L323I
25719,TCGA-CZ-4853-01A-01D-1429-08,SSX5,missense_variant,p.F70L
25720,TCGA-CZ-4853-01A-01D-1429-08,SHROOM4,synonymous_variant,p.V11V


In [79]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'KIRC'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'KIRC'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

# 336개

100%|██████████████████████████████████████████████████████████████████████████████| 1051/1051 [07:27<00:00,  2.35it/s]


# additional data

# LAML

In [80]:
import os
import tarfile
import pandas as pd

# KIRC 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/LAML'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                
                # df가 None이 아닌 경우에만 계속 처리
                if df is not None:
                    # Amino_Acid_Change 열이 있는지 확인하고, 있으면 이름 변경
                    if 'Amino_Acid_Change' in df.columns:
                        df.rename(columns={'Amino_Acid_Change': 'HGVSp_Short'}, inplace=True)
                    
                    # 필요한 열만 선택
                    df = df[['Tumor_Sample_Barcode', 'Hugo_Symbol', 'Consequence', 'HGVSp_Short']]
                    
                    # df_list에 추가
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출

                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/LAML.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\2214348820.py:18: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\2214348820.py:18: DtypeWarning: Columns (1,2,5,6,15,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,42,51,57,58,63,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


'#version' detected with skiprows=0, retrying with skiprows=1


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\2214348820.py:18: DtypeWarning: Columns (1,2,5,6,15,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,42,51,57,58,63,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


'#version' detected with skiprows=1, retrying with skiprows=2


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\2214348820.py:18: DtypeWarning: Columns (12,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


ParserError encountered with skiprows=0, retrying with skiprows=2


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\2214348820.py:18: DtypeWarning: Columns (33,34,35,36,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


In [81]:
df_TCGA

,Tumor_Sample_Barcode,Hugo_Symbol,Consequence,HGVSp_Short
0,aml_ohsu_2018_14-00141,C1orf170,missense_variant,p.A551T
2,aml_ohsu_2018_15-00492,C1orf170,missense_variant,p.A396V
4,aml_ohsu_2018_14-00676,AGRN,missense_variant,p.G1111V
6,aml_ohsu_2018_13-00515,UBE2J2,missense_variant,p.M96L
7,aml_ohsu_2018_15-00805,CPSF3L,missense_variant,p.V118I
...,...,...,...,...
3895,TCGA-AB-2956-03A-01W-0733-08,POLR2B,missense_variant,p.I710V
3896,TCGA-AB-2956-03A-01W-0733-08,RWDD4,synonymous_variant,p.L11L
3897,TCGA-AB-2956-03A-01W-0733-08,GRM6,synonymous_variant,p.G226G
3898,TCGA-AB-2956-03A-01W-0733-08,PKP2,stop_gained,p.R147*


In [82]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'LAML'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'LAML'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|██████████████████████████████████████████████████████████████████████████████| 1590/1590 [11:50<00:00,  2.24it/s]


# additional data

# LGG

In [83]:
import os
import tarfile
import pandas as pd

# KIRC 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/LGG'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                
                # df가 None이 아닌 경우에만 계속 처리
                if df is not None:
                    # Amino_Acid_Change 열이 있는지 확인하고, 있으면 이름 변경
                    if 'Amino_Acid_Change' in df.columns:
                        df.rename(columns={'Amino_Acid_Change': 'HGVSp_Short'}, inplace=True)
                    
                    # 필요한 열만 선택
                    df = df[['Tumor_Sample_Barcode', 'Hugo_Symbol', 'Consequence', 'HGVSp_Short']]
                    
                    # df_list에 추가
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출

                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/LGG.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\898209436.py:18: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\898209436.py:73: DtypeWarning: Columns (74,78,112,114,120,130,131) have mixed types. Specify dtype option on import or set low_memory=False.
  df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분


In [84]:
df_TCGA

,Tumor_Sample_Barcode,Hugo_Symbol,Consequence,HGVSp_Short
0,TCGA-DU-6394-01,NOS1,synonymous_variant,p.T1128T
2,TCGA-DU-6394-01,JAG2,synonymous_variant,p.N547N
3,TCGA-DU-6394-01,ENKD1,missense_variant,p.R72H
4,TCGA-DU-6394-01,FKBP8,synonymous_variant,p.T168T
5,TCGA-DU-6394-01,IRF2BP1,missense_variant,p.C552F
...,...,...,...,...
32774,TCGA-P5-A77W-01A-11D-A32B-08,CCDC198,missense_variant,p.Q183P
32776,TCGA-P5-A77W-01A-11D-A32B-08,PLVAP,missense_variant,p.R19Q
32777,TCGA-P5-A77W-01A-11D-A32B-08,CIC,frameshift_variant,p.V760Hfs
32778,TCGA-P5-A77W-01A-11D-A32B-08,PHKA2,missense_variant,p.R687H


In [85]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'LGG'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'LGG'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|██████████████████████████████████████████████████████████████████████████████| 1029/1029 [08:06<00:00,  2.12it/s]


In [86]:
train

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R895R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18928,ID,LGG,WT,WT,WT,WT,WT,WT,WT,WT,...,R195G,WT,WT,WT,WT,WT,WT,WT,WT,WT
18929,ID,LGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
18930,ID,LGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
18931,ID,LGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


# additional data

# LIHC

In [87]:
import os
import tarfile
import pandas as pd

# KIRC 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/LIHC'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                
                # df가 None이 아닌 경우에만 계속 처리
                if df is not None:
                    # Amino_Acid_Change 열이 있는지 확인하고, 있으면 이름 변경
                    if 'Amino_Acid_Change' in df.columns:
                        df.rename(columns={'Amino_Acid_Change': 'HGVSp_Short'}, inplace=True)
                    
                    # 필요한 열만 선택
                    df = df[['Tumor_Sample_Barcode', 'Hugo_Symbol', 'Consequence', 'HGVSp_Short']]
                    
                    # df_list에 추가
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출

                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/LIHC.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\974844445.py:18: DtypeWarning: Columns (1,20,21,22,23,111) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


'#version' detected with skiprows=0, retrying with skiprows=1


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\974844445.py:18: DtypeWarning: Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


In [88]:
df_TCGA

,Tumor_Sample_Barcode,Hugo_Symbol,Consequence,HGVSp_Short
0,H050150,ATP4B,missense_variant,p.C152F
1,H050150,GNAT2,missense_variant,p.A111T
2,H050150,CLDN25,missense_variant,p.A200T
3,H050150,RADIL,missense_variant,p.A437T
4,H050150,CTNND2,missense_variant,p.S611T
...,...,...,...,...
45434,TCGA-2Y-A9GW-01A-11D-A382-10,CD79A,synonymous_variant,p.T146T
45435,TCGA-2Y-A9GW-01A-11D-A382-10,LILRA4,synonymous_variant,p.L464L
45436,TCGA-2Y-A9GW-01A-11D-A382-10,ZNF772,missense_variant,p.E117G
45438,TCGA-2Y-A9GW-01A-11D-A382-10,SHANK3,synonymous_variant,p.G183G


In [89]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'LIHC'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'LIHC'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|████████████████████████████████████████████████████████████████████████████████| 986/986 [08:10<00:00,  2.01it/s]


# additional data

# LUAD

In [90]:
import os
import tarfile
import pandas as pd

# KIRC 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/LUAD'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                
                # df가 None이 아닌 경우에만 계속 처리
                if df is not None:
                    # Amino_Acid_Change 열이 있는지 확인하고, 있으면 이름 변경
                    
                    if 'Amino_Acid_Change' in df.columns and 'HGVSp_Short' in df.columns:
                        df.drop(columns=['Amino_Acid_Change'], inplace=True)
                        
                    if 'Amino_Acid_Change' in df.columns:
                        df.rename(columns={'Amino_Acid_Change': 'HGVSp_Short'}, inplace=True)
                    
                    # 필요한 열만 선택
                    df = df[['Tumor_Sample_Barcode', 'Hugo_Symbol', 'Consequence', 'HGVSp_Short']]
                    
                    # df_list에 추가
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출

                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/LUAD.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\888325987.py:18: DtypeWarning: Columns (4,15,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


ParserError encountered with skiprows=0, retrying with skiprows=2


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\888325987.py:18: DtypeWarning: Columns (131) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


ParserError encountered with skiprows=0, retrying with skiprows=2
ParserError encountered with skiprows=0, retrying with skiprows=2
ParserError encountered with skiprows=0, retrying with skiprows=2


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\888325987.py:18: DtypeWarning: Columns (50,57,59,61,71,95,96) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


ParserError encountered with skiprows=0, retrying with skiprows=2


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\888325987.py:18: DtypeWarning: Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\888325987.py:18: DtypeWarning: Columns (65,207) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


In [91]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'LUAD'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'LUAD'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|██████████████████████████████████████████████████████████████████████████████| 3654/3654 [33:56<00:00,  1.79it/s]


In [92]:
train

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R895R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23568,ID,LUAD,G1310W,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,V83V,WT,WT,WT
23569,ID,LUAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
23570,ID,LUAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
23571,ID,LUAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,E216K,WT,WT,WT,WT


# additional data

# LUSC

In [93]:
import os
import tarfile
import pandas as pd

# KIRC 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/LUSC'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                
                # df가 None이 아닌 경우에만 계속 처리
                if df is not None:
                    # Amino_Acid_Change 열이 있는지 확인하고, 있으면 이름 변경
                    
                    if 'Amino_Acid_Change' in df.columns and 'HGVSp_Short' in df.columns:
                        df.drop(columns=['Amino_Acid_Change'], inplace=True)
                        
                    if 'Amino_Acid_Change' in df.columns:
                        df.rename(columns={'Amino_Acid_Change': 'HGVSp_Short'}, inplace=True)
                    
                    # 필요한 열만 선택
                    df = df[['Tumor_Sample_Barcode', 'Hugo_Symbol', 'Consequence', 'HGVSp_Short']]
                    
                    # df_list에 추가
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출

                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/LUSC.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1832731999.py:18: DtypeWarning: Columns (1,5,6,15,20,21,22,23,24,25,26,30,31,45,46,47,48,49,50,58,60,61,63,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,88,91,92,95,96,97,98,100,103,106,111,117,118,119,120,121,122,123,124,125,126,127,129) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


'#version' detected with skiprows=0, retrying with skiprows=1


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1832731999.py:18: DtypeWarning: Columns (1,5,6,15,20,21,22,23,24,25,26,30,31,45,46,47,48,49,50,58,60,61,63,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,88,91,92,95,96,97,98,100,103,106,111,117,118,119,120,121,122,123,124,125,126,127,129) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


'#version' detected with skiprows=1, retrying with skiprows=2
ParserError encountered with skiprows=0, retrying with skiprows=2
ParserError encountered with skiprows=0, retrying with skiprows=2


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1832731999.py:18: DtypeWarning: Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1832731999.py:18: DtypeWarning: Columns (1,5,6,20,21,22,23,24,26,30,31,42,44,45,46,47,50,54,55,57,62,67,70,71,72,73,89,95,98,100,101,103,104,108,110,112,113,117,124,126,127,130,136,137,138,144,145,147,149,150,152,155,160,161,165,171,172,178,182,184,191,199,201,214,215,223,228,230,235) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


'#version' detected with skiprows=0, retrying with skiprows=1


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1832731999.py:18: DtypeWarning: Columns (73,101,124,138,149,165) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1832731999.py:77: DtypeWarning: Columns (114) have mixed types. Specify dtype option on import or set low_memory=False.
  df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분


In [94]:
df_TCGA

,Tumor_Sample_Barcode,Hugo_Symbol,Consequence,HGVSp_Short
0,C3L-00081,FBN2,frameshift_variant,p.I1644Sfs
1,C3L-00081,HNF4G,frameshift_variant,p.M314Wfs
2,C3L-00081,SNRPN,frameshift_variant,p.G102Efs
3,C3L-00081,EIF2AK4,frameshift_variant,p.V564Yfs
4,C3L-00081,GSDMA,frameshift_variant,p.G67Afs
...,...,...,...,...
172821,TCGA-66-2785-01A-01D-1522-08,TENM1,missense_variant,p.A120G
172822,TCGA-66-2785-01A-01D-1522-08,OCRL,synonymous_variant,p.L209L
172823,TCGA-66-2785-01A-01D-1522-08,MAGEC2,missense_variant,p.G74V
172824,TCGA-66-2785-01A-01D-1522-08,F8,missense_variant,p.E871Q


In [95]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'LUSC'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'LUSC'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|██████████████████████████████████████████████████████████████████████████████| 1702/1702 [17:38<00:00,  1.61it/s]


# additional data

# OV

In [96]:
import os
import tarfile
import pandas as pd

# KIRC 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/OV'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                
                # df가 None이 아닌 경우에만 계속 처리
                if df is not None:
                    # Amino_Acid_Change 열이 있는지 확인하고, 있으면 이름 변경
                    
                    if 'Amino_Acid_Change' in df.columns and 'HGVSp_Short' in df.columns:
                        df.drop(columns=['Amino_Acid_Change'], inplace=True)
                        
                    if 'Amino_Acid_Change' in df.columns:
                        df.rename(columns={'Amino_Acid_Change': 'HGVSp_Short'}, inplace=True)
                    
                    # 필요한 열만 선택
                    df = df[['Tumor_Sample_Barcode', 'Hugo_Symbol', 'Consequence', 'HGVSp_Short']]
                    
                    # df_list에 추가
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출

                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/OV.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1074750026.py:18: DtypeWarning: Columns (1,5,6,20,21,22,23,30,31,33,34,35,36,42,44,45,46,48,55,56,57,59,60,65,66,71,72,73,74,83,91,103,105,106,108,115,119,123,124,126,128) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


'#version' detected with skiprows=0, retrying with skiprows=1


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1074750026.py:18: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


In [97]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'OV'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'OV'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|████████████████████████████████████████████████████████████████████████████████| 964/964 [10:24<00:00,  1.54it/s]


# additional data

# PAAD

In [98]:
import os
import tarfile
import pandas as pd

# KIRC 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/PAAD'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                
                # df가 None이 아닌 경우에만 계속 처리
                if df is not None:
                    # Amino_Acid_Change 열이 있는지 확인하고, 있으면 이름 변경
                    
                    if 'Amino_Acid_Change' in df.columns and 'HGVSp_Short' in df.columns:
                        df.drop(columns=['Amino_Acid_Change'], inplace=True)
                        
                    if 'Amino_Acid_Change' in df.columns:
                        df.rename(columns={'Amino_Acid_Change': 'HGVSp_Short'}, inplace=True)
                    
                    # 필요한 열만 선택
                    df = df[['Tumor_Sample_Barcode', 'Hugo_Symbol', 'Consequence', 'HGVSp_Short']]
                    
                    # df_list에 추가
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출

                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/PAAD.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1577363837.py:18: DtypeWarning: Columns (4,51,94,135,172,243) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


ParserError encountered with skiprows=0, retrying with skiprows=2


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1577363837.py:18: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1577363837.py:77: DtypeWarning: Columns (120) have mixed types. Specify dtype option on import or set low_memory=False.
  df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분


In [99]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'PAAD'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'PAAD'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|████████████████████████████████████████████████████████████████████████████████| 960/960 [10:43<00:00,  1.49it/s]


# additional data

# PCPG

In [100]:
import os
import tarfile
import pandas as pd

# KIRC 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/PCPG'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                
                # df가 None이 아닌 경우에만 계속 처리
                if df is not None:
                    # Amino_Acid_Change 열이 있는지 확인하고, 있으면 이름 변경
                    
                    if 'Amino_Acid_Change' in df.columns and 'HGVSp_Short' in df.columns:
                        df.drop(columns=['Amino_Acid_Change'], inplace=True)
                        
                    if 'Amino_Acid_Change' in df.columns:
                        df.rename(columns={'Amino_Acid_Change': 'HGVSp_Short'}, inplace=True)
                    
                    # 필요한 열만 선택
                    df = df[['Tumor_Sample_Barcode', 'Hugo_Symbol', 'Consequence', 'HGVSp_Short']]
                    
                    # df_list에 추가
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출

                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/PCPG.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\2915514043.py:18: DtypeWarning: Columns (63,224) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


In [101]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'PCPG'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'PCPG'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|████████████████████████████████████████████████████████████████████████████████| 343/343 [03:51<00:00,  1.48it/s]


# additional data

# PRAD

In [102]:
import os
import tarfile
import pandas as pd

# KIRC 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/PRAD'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                
                # df가 None이 아닌 경우에만 계속 처리
                if df is not None:
                    # Amino_Acid_Change 열이 있는지 확인하고, 있으면 이름 변경
                    
                    if 'Amino_Acid_Change' in df.columns and 'HGVSp_Short' in df.columns:
                        df.drop(columns=['Amino_Acid_Change'], inplace=True)
                        
                    if 'Amino_Acid_Change' in df.columns:
                        df.rename(columns={'Amino_Acid_Change': 'HGVSp_Short'}, inplace=True)
                    
                    # 필요한 열만 선택
                    df = df[['Tumor_Sample_Barcode', 'Hugo_Symbol', 'Consequence', 'HGVSp_Short']]
                    
                    # df_list에 추가
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출

                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/PRAD.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

'#version' detected with skiprows=0, retrying with skiprows=1


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1622103289.py:18: DtypeWarning: Columns (103,235) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


'#version' detected with skiprows=0, retrying with skiprows=1
'#version' detected with skiprows=1, retrying with skiprows=2
'#version' detected with skiprows=0, retrying with skiprows=1


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1622103289.py:18: DtypeWarning: Columns (100) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1622103289.py:18: DtypeWarning: Columns (67) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1622103289.py:18: DtypeWarning: Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1622103289.py:18: DtypeWarning: Columns (1,5,6,20,21,22,23,24,25,30,31,33,34,35,36,42,44,45,46,48,53,54,55,56,57,61,64,65,66,67,73,79,87,89,90,92,97,100,104,106,108) have mixed types. Specify dtype option on import or set low_memory

'#version' detected with skiprows=0, retrying with skiprows=1


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1622103289.py:77: DtypeWarning: Columns (120) have mixed types. Specify dtype option on import or set low_memory=False.
  df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분


In [103]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'PRAD'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'PRAD'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|██████████████████████████████████████████████████████████████████████████████| 3980/3980 [47:50<00:00,  1.39it/s]


In [104]:
train

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R895R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31517,ID,PRAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
31518,ID,PRAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
31519,ID,PRAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
31520,ID,PRAD,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


# additional data

# SARC

In [105]:
import os
import tarfile
import pandas as pd

# KIRC 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/SARC'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                
                # df가 None이 아닌 경우에만 계속 처리
                if df is not None:
                    # Amino_Acid_Change 열이 있는지 확인하고, 있으면 이름 변경
                    
                    if 'Amino_Acid_Change' in df.columns and 'HGVSp_Short' in df.columns:
                        df.drop(columns=['Amino_Acid_Change'], inplace=True)
                        
                    if 'Amino_Acid_Change' in df.columns:
                        df.rename(columns={'Amino_Acid_Change': 'HGVSp_Short'}, inplace=True)
                    
                    # 필요한 열만 선택
                    df = df[['Tumor_Sample_Barcode', 'Hugo_Symbol', 'Consequence', 'HGVSp_Short']]
                    
                    # df_list에 추가
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출

                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/SARC.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

In [106]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'SARC'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'SARC'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|██████████████████████████████████████████████████████████████████████████████| 1803/1803 [23:05<00:00,  1.30it/s]


# additional data

# SKCM

In [107]:
import os
import tarfile
import pandas as pd

# KIRC 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/SKCM'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                
                # df가 None이 아닌 경우에만 계속 처리
                if df is not None:
                    # Amino_Acid_Change 열이 있는지 확인하고, 있으면 이름 변경
                    
                    if 'Amino_Acid_Change' in df.columns and 'HGVSp_Short' in df.columns:
                        df.drop(columns=['Amino_Acid_Change'], inplace=True)
                        
                    if 'Amino_Acid_Change' in df.columns:
                        df.rename(columns={'Amino_Acid_Change': 'HGVSp_Short'}, inplace=True)
                    
                    # 필요한 열만 선택
                    df = df[['Tumor_Sample_Barcode', 'Hugo_Symbol', 'Consequence', 'HGVSp_Short']]
                    
                    # df_list에 추가
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출

                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/SKCM.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\4222089456.py:18: DtypeWarning: Columns (1,4,5,6,20,21,22,23,24,25,26,27,28,29,30,31,42,44,45,46,48,53,54,55,56,57,61,64,65,66,67,73,74,80,88,90,91,93,96,99,102,107,110,112,113) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


'#version' detected with skiprows=0, retrying with skiprows=1


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\4222089456.py:18: DtypeWarning: Columns (4,53,64,90,113) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\4222089456.py:18: DtypeWarning: Columns (46,67,89) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\4222089456.py:18: DtypeWarning: Columns (4,38,39,43,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\4222089456.py:18: DtypeWarning: Columns (4,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


In [108]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'SKCM'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'SKCM'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|██████████████████████████████████████████████████████████████████████████████| 1198/1198 [16:29<00:00,  1.21it/s]


In [356]:
train

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R895R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26754,ID,SKCM,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
26755,ID,SKCM,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
26756,ID,SKCM,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
26757,ID,SKCM,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


# additional data

# TGCT

In [109]:
import os
import tarfile
import pandas as pd

# KIRC 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/TGCT'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                
                # df가 None이 아닌 경우에만 계속 처리
                if df is not None:
                    # Amino_Acid_Change 열이 있는지 확인하고, 있으면 이름 변경
                    
                    if 'Amino_Acid_Change' in df.columns and 'HGVSp_Short' in df.columns:
                        df.drop(columns=['Amino_Acid_Change'], inplace=True)
                        
                    if 'Amino_Acid_Change' in df.columns:
                        df.rename(columns={'Amino_Acid_Change': 'HGVSp_Short'}, inplace=True)
                    
                    # 필요한 열만 선택
                    df = df[['Tumor_Sample_Barcode', 'Hugo_Symbol', 'Consequence', 'HGVSp_Short']]
                    
                    # df_list에 추가
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출

                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/TGCT.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

In [110]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'TGCT'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'TGCT'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|████████████████████████████████████████████████████████████████████████████████| 295/295 [04:08<00:00,  1.18it/s]


# additional data

# THCA

In [111]:
import os
import tarfile
import pandas as pd

# KIRC 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/THCA'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                
                # df가 None이 아닌 경우에만 계속 처리
                if df is not None:
                    # Amino_Acid_Change 열이 있는지 확인하고, 있으면 이름 변경
                    
                    if 'Amino_Acid_Change' in df.columns and 'HGVSp_Short' in df.columns:
                        df.drop(columns=['Amino_Acid_Change'], inplace=True)
                        
                    if 'Amino_Acid_Change' in df.columns:
                        df.rename(columns={'Amino_Acid_Change': 'HGVSp_Short'}, inplace=True)
                    
                    # 필요한 열만 선택
                    df = df[['Tumor_Sample_Barcode', 'Hugo_Symbol', 'Consequence', 'HGVSp_Short']]
                    
                    # df_list에 추가
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출

                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/THCA.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

In [112]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'THCA'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'THCA'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [13:13<00:00,  1.18it/s]


# additional data

# THYM

In [113]:
import os
import tarfile
import pandas as pd

# KIRC 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/THYM'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                
                # df가 None이 아닌 경우에만 계속 처리
                if df is not None:
                    # Amino_Acid_Change 열이 있는지 확인하고, 있으면 이름 변경
                    
                    if 'Amino_Acid_Change' in df.columns and 'HGVSp_Short' in df.columns:
                        df.drop(columns=['Amino_Acid_Change'], inplace=True)
                        
                    if 'Amino_Acid_Change' in df.columns:
                        df.rename(columns={'Amino_Acid_Change': 'HGVSp_Short'}, inplace=True)
                    
                    # 필요한 열만 선택
                    df = df[['Tumor_Sample_Barcode', 'Hugo_Symbol', 'Consequence', 'HGVSp_Short']]
                    
                    # df_list에 추가
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출

                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/THYM.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

ParserError encountered with skiprows=0, retrying with skiprows=2


In [114]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'THYM'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'THYM'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|████████████████████████████████████████████████████████████████████████████████| 323/323 [04:35<00:00,  1.17it/s]


# additional data

# UCEC

In [115]:
import os
import tarfile
import pandas as pd

# KIRC 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/UCEC'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                
                # df가 None이 아닌 경우에만 계속 처리
                if df is not None:
                    # Amino_Acid_Change 열이 있는지 확인하고, 있으면 이름 변경
                    
                    if 'Amino_Acid_Change' in df.columns and 'HGVSp_Short' in df.columns:
                        df.drop(columns=['Amino_Acid_Change'], inplace=True)
                        
                    if 'Amino_Acid_Change' in df.columns:
                        df.rename(columns={'Amino_Acid_Change': 'HGVSp_Short'}, inplace=True)
                    
                    # 필요한 열만 선택
                    df = df[['Tumor_Sample_Barcode', 'Hugo_Symbol', 'Consequence', 'HGVSp_Short']]
                    
                    # df_list에 추가
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출

                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/UCEC.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\2142324066.py:18: DtypeWarning: Columns (4,38,39,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\2142324066.py:18: DtypeWarning: Columns (1,4,5,6,20,21,22,23,29,30,33,34,35,36,42,44,45,46,48,53,54,55,57,58,63,67,68,69,70,72,76,80,87,94,95,99,101,102,104,109,112,116,117,121,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


'#version' detected with skiprows=0, retrying with skiprows=1


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\2142324066.py:18: DtypeWarning: Columns (125) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\2142324066.py:77: DtypeWarning: Columns (74,114,120) have mixed types. Specify dtype option on import or set low_memory=False.
  df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분


In [116]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'UCEC'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'UCEC'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|██████████████████████████████████████████████████████████████████████████████| 1036/1036 [15:46<00:00,  1.09it/s]


In [366]:
train

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R895R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28126,ID,UCEC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
28127,ID,UCEC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,H233H,WT,WT,Q220*,WT
28128,ID,UCEC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
28129,ID,UCEC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


# KIPAN, STES 찾아봐야됌

# additional data

# KICH(KIPAN)

In [117]:
import os
import tarfile
import pandas as pd

# KIRC 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/KICH'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                
                # df가 None이 아닌 경우에만 계속 처리
                if df is not None:
                    # Amino_Acid_Change 열이 있는지 확인하고, 있으면 이름 변경
                    
                    if 'Amino_Acid_Change' in df.columns and 'HGVSp_Short' in df.columns:
                        df.drop(columns=['Amino_Acid_Change'], inplace=True)
                        
                    if 'Amino_Acid_Change' in df.columns:
                        df.rename(columns={'Amino_Acid_Change': 'HGVSp_Short'}, inplace=True)
                    
                    # 필요한 열만 선택
                    df = df[['Tumor_Sample_Barcode', 'Hugo_Symbol', 'Consequence', 'HGVSp_Short']]
                    
                    # df_list에 추가
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출

                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/KICH.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

'#version' detected with skiprows=0, retrying with skiprows=1


In [118]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'KICH'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'KICH'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|████████████████████████████████████████████████████████████████████████████████| 131/131 [01:57<00:00,  1.12it/s]


# additional data

# KIRP(KIPAN)

In [119]:
import os
import tarfile
import pandas as pd

# KIRC 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/KIRP'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                
                # df가 None이 아닌 경우에만 계속 처리
                if df is not None:
                    # Amino_Acid_Change 열이 있는지 확인하고, 있으면 이름 변경
                    
                    if 'Amino_Acid_Change' in df.columns and 'HGVSp_Short' in df.columns:
                        df.drop(columns=['Amino_Acid_Change'], inplace=True)
                        
                    if 'Amino_Acid_Change' in df.columns:
                        df.rename(columns={'Amino_Acid_Change': 'HGVSp_Short'}, inplace=True)
                    
                    # 필요한 열만 선택
                    df = df[['Tumor_Sample_Barcode', 'Hugo_Symbol', 'Consequence', 'HGVSp_Short']]
                    
                    # df_list에 추가
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출

                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/KIRP.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\4184964100.py:18: DtypeWarning: Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\4184964100.py:77: DtypeWarning: Columns (120) have mixed types. Specify dtype option on import or set low_memory=False.
  df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분


In [120]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'KIRP'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'KIRP'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|████████████████████████████████████████████████████████████████████████████████| 564/564 [08:29<00:00,  1.11it/s]


# additional data

# ESCA(STES)

In [121]:
import os
import tarfile
import pandas as pd

# KIRC 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/ESCA'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                
                # df가 None이 아닌 경우에만 계속 처리
                if df is not None:
                    # Amino_Acid_Change 열이 있는지 확인하고, 있으면 이름 변경
                    
                    if 'Amino_Acid_Change' in df.columns and 'HGVSp_Short' in df.columns:
                        df.drop(columns=['Amino_Acid_Change'], inplace=True)
                        
                    if 'Amino_Acid_Change' in df.columns:
                        df.rename(columns={'Amino_Acid_Change': 'HGVSp_Short'}, inplace=True)
                    
                    # 필요한 열만 선택
                    df = df[['Tumor_Sample_Barcode', 'Hugo_Symbol', 'Consequence', 'HGVSp_Short']]
                    
                    # df_list에 추가
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출

                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/ESCA.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

In [122]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()

tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'ESCA'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'ESCA'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|████████████████████████████████████████████████████████████████████████████████| 714/714 [11:03<00:00,  1.08it/s]


# additional data

# STAD(STES)

In [123]:
import os
import tarfile
import pandas as pd

# KIRC 폴더에 있는 모든 .tar.gz 파일을 처리
folder_path = 'additional_data_2/STAD'
gz_files = [f for f in os.listdir(folder_path) if f.endswith('.tar.gz')]

# 데이터를 저장할 리스트
df_list = []

# DataFrame을 로드하는 함수 (skiprows 조정 기능 포함)
def load_dataframe(file, sep='\t'):
    skiprows = 0  # 처음에는 skiprows=0으로 시도
    
    while True:
        try:
            df = pd.read_csv(file, sep=sep, skiprows=skiprows)
            # 첫 번째 열 이름이 '#version'으로 시작하는지 확인
            if df.columns[0].startswith('#'):
                print(f"'#version' detected with skiprows={skiprows}, retrying with skiprows={skiprows+1}")
                skiprows += 1  # skiprows 값을 1 증가시키고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                break  # '#version'이 없으면 루프 탈출
        except pd.errors.ParserError:
            if skiprows == 0:
                print(f"ParserError encountered with skiprows=0, retrying with skiprows=2")
                skiprows = 2  # skiprows를 2로 설정하고 다시 시도
                file.seek(0)  # 파일 포인터를 다시 시작점으로 이동
            else:
                print(f"ParserError encountered again with skiprows={skiprows}. Skipping file.")
                return None  # 에러가 발생하면 None 반환
    return df

# 각 tar.gz 파일을 순차적으로 처리
for gz_file in gz_files:
    file_path = os.path.join(folder_path, gz_file)
    
    with tarfile.open(file_path, 'r:gz') as tar:
        # 압축 파일 내부의 파일 목록을 확인
        file_names = tar.getnames()
        
        # 우리가 필요한 파일 경로를 찾아서 추출
        for file_name in file_names:
            if 'data_mutations.txt' in file_name:  # 필요한 파일의 경로를 찾음
                file = tar.extractfile(file_name)
                
                # DataFrame을 로드하는 함수 호출
                df = load_dataframe(file, sep='\t')
                
                # df가 None이 아닌 경우에만 계속 처리
                if df is not None:
                    # Amino_Acid_Change 열이 있는지 확인하고, 있으면 이름 변경
                    
                    if 'Amino_Acid_Change' in df.columns and 'HGVSp_Short' in df.columns:
                        df.drop(columns=['Amino_Acid_Change'], inplace=True)
                        
                    if 'Amino_Acid_Change' in df.columns:
                        df.rename(columns={'Amino_Acid_Change': 'HGVSp_Short'}, inplace=True)
                    
                    # 필요한 열만 선택
                    df = df[['Tumor_Sample_Barcode', 'Hugo_Symbol', 'Consequence', 'HGVSp_Short']]
                    
                    # df_list에 추가
                    df_list.append(df)
                break  # 필요한 파일만 처리하므로 루프 탈출

                
df_TCGA = pd.concat(df_list, ignore_index=True)
###################################
# .maf.gz 파일을 불러오기
file_path_1 = 'additional_data/STAD.gz'

# 파일을 pandas의 DataFrame으로 읽기
with gzip.open(file_path_1, 'rt') as f:
    df_TCGA_2 = pd.read_csv(f, sep='\t', comment='#')  # 주석(#) 무시하고 탭으로 구분
    
df_TCGA_2= df_TCGA_2[['Tumor_Sample_Barcode','Hugo_Symbol','Consequence','HGVSp_Short']]
df_TCGA = pd.concat([df_TCGA,df_TCGA_2])
#####################################
df_TCGA = df_TCGA.dropna(subset='HGVSp_Short')
# 'frameshift_variant'이고, 'HGVSp_Short' 열에 'fs'가 없는 값들을 'fs'를 추가하여 수정합니다.
df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'] = \
    df_TCGA.loc[(df_TCGA['Consequence'] == 'frameshift_variant') & ~df_TCGA['HGVSp_Short'].str.contains('fs'), 'HGVSp_Short'].str.replace(r'(\*)$', r'fs\1', regex=True)

# 'Variant_Classification'이 "Silent"인 행의 'HGVSp_Short' 열 값을 수정합니다.
df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'] = \
    df_TCGA.loc[df_TCGA['Consequence'] == "synonymous_variant", 'HGVSp_Short'].str.replace(r'(\w)(\d+)=', r'\1\2\1', regex=True)

df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'] = df_TCGA.loc[df_TCGA['Consequence'] == 'frameshift_variant', 'HGVSp_Short'].str.split('*').str[0]
df_TCGA = df_TCGA[df_TCGA['Consequence'].isin(['missense_variant', 'synonymous_variant', 'stop_gained', 'frameshift_variant','inframe_deletion','inframe_insertion'])]
df_TCGA = df_TCGA.drop_duplicates()

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1723642408.py:18: DtypeWarning: Columns (56,61,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


ParserError encountered with skiprows=0, retrying with skiprows=2


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1723642408.py:18: DtypeWarning: Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_31332\1723642408.py:18: DtypeWarning: Columns (51,81,89) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep=sep, skiprows=skiprows)


In [124]:
lst = train.drop(columns=['ID','SUBCLASS']).columns
train_col_list = list(lst)
TCGA = df_TCGA[df_TCGA['Hugo_Symbol'].isin(train_col_list)]
tcga_data = TCGA[['Tumor_Sample_Barcode','Hugo_Symbol','HGVSp_Short']].dropna()



tcga_data['ID'] = tcga_data['Tumor_Sample_Barcode']
tcga_data['HGVSp_Short'] = tcga_data['HGVSp_Short'].str[2:]
tcga_data['SUBCLASS'] = 'STAD'
tcga_data = tcga_data.drop(columns=['Tumor_Sample_Barcode'])

ID_unique = tcga_data['ID'].unique()

for idx in tqdm(ID_unique):
    data = tcga_data[tcga_data['ID']==idx].groupby('Hugo_Symbol')['HGVSp_Short'].agg(lambda x: ' '.join(x)).reset_index()
    data['HGVSp_Short'] = data['HGVSp_Short'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))

    new_row = pd.Series(['WT'] * train.shape[1], index=train.columns)
    new_row['ID'] = 'ID'
    new_row['SUBCLASS'] = 'STAD'

        # Hugo_Symbol과 base_series의 인덱스 이름이 같으면 WT 값을 HGVSp_Short로 변경
    for index, row in data.iterrows():
        hugo_symbol = row['Hugo_Symbol']
        mutation = row['HGVSp_Short']

        # new_row의 인덱스 중 Hugo_Symbol과 일치하는 항목을 찾고 'WT'를 변이 값으로 교체
        if hugo_symbol in new_row.index:
            new_row[hugo_symbol] = mutation

    train = pd.concat([train, pd.DataFrame(new_row).T], axis=0)

train = train.reset_index(drop=True)

100%|██████████████████████████████████████████████████████████████████████████████| 1444/1444 [23:23<00:00,  1.03it/s]


In [127]:
train

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R895R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39963,ID,STAD,M713I,Q456Sfs,WT,WT,R220R A304T,L2012P,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
39964,ID,STAD,C48R,WT,WT,WT,WT,WT,WT,R1305W,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
39965,ID,STAD,WT,WT,WT,WT,WT,K2231K,WT,WT,...,WT,WT,WT,WT,WT,S354L,WT,WT,WT,WT
39966,ID,STAD,I822I F229L,WT,WT,WT,WT,E2204D S1024N,WT,N553N L276F,...,R1397* R2115*,WT,K51T K174T,WT,L465I,WT,H335Y E275D,V450G,WT,V524A


In [125]:
# train.to_csv('1019_final_tr_1.csv', encoding='UTF-8-sig', index=False)
# test.to_csv('1019_final_te_1.csv', encoding='UTF-8-sig', index=False)

In [126]:
train = train.fillna('WT')
test = test.fillna('WT')

train = train.applymap(convert_format)
train = train.applymap(convert_deletion_format)

test = test.applymap(convert_format)
test = test.applymap(convert_deletion_format)

train.to_csv('1019_final_tr_2.csv', encoding='UTF-8-sig', index=False)
test.to_csv('1019_final_te_2.csv', encoding='UTF-8-sig', index=False)

# UNTIL HERE